In [1]:
%load_ext line_profiler

## This is a code to debug the configuration file that leads to error in IOManager in larcv3. 

In [2]:
import larcv

In [3]:
import os,sys,json

In [4]:
print(os.listdir("./data/"))

['merged_beam_0.h5', 'Untitled1.ipynb', 'Untitled.ipynb', 'merged_beam_0.pkl', 'look_hdf5_files.ipynb', 'merged_sample_0.h5', 'voxel_occupancy.ipynb', 'read_pickle.ipynb', 'convert_to_pkl.ipynb', '.ipynb_checkpoints', 'test_torch.ipynb']


In [5]:
import h5py

In [6]:
f = "./data/merged_sample_0.h5"

In [7]:
io = larcv.IOManager()

In [8]:
#io.add_in_file(f)
#io.initialize()

In [9]:
io_config = {'filler_name': 'primary', 'filler_cfg': 'tmpsbbj5ext', 'verbosity': 5, 'make_copy': False}
data_keys = {'image': 'data', 'label_neut': 'label_neut', 'label_prot': 'label_prot', 'label_cpi': 'label_cpi', 'label_npi': 'label_npi'}


In [10]:
class queue_interface(object):

    def __init__(self, verbose=False, random_access_mode="random_blocks", seed=None):
        '''init function

        Not much to store here, just a dict of dataloaders and the keys to access their data.

        Queue loaders are manually triggered IO, not always running, so
        '''
        object.__init__(self)
        self._queueloaders  = {}
        self._data_keys     = {}
        self._dims          = {}
        self._verbose       = verbose
        self._random_access = RandomAccess[random_access_mode]
        self._minibatch_size = {}

        self._queue_prev_entries = {}
        self._queue_next_entries = {}
        self._count = {}

        if seed is not None:
            random.seed(seed)
            numpy.random.seed(seed)

        self._warning = True

        self._writer = None

    def no_warnings(self):
        self._warning = False

    def get_next_batch_indexes(self, mode, minibatch_size):

        # Using the random_access parameter, determine which entries to read:

        # In this function, we promote the "next" to "prev".  Do that first:
        if mode not in self._queue_next_entries:
            self._queue_next_entries[mode] = None
        self._queue_prev_entries[mode] = self._queue_next_entries[mode]
        # (If this has not been run, they will both be set to None)

        if self._random_access == RandomAccess.serial_access:
            # Figure out the last batch's highest entry:
            if mode in self._queue_prev_entries:
                if self._queue_prev_entries[mode] is not None:
                    last_entry = numpy.max(self._queue_prev_entries[mode])
                else:
                    last_entry = -1
            else:
                last_entry = -1

            next_last_entry = minibatch_size + last_entry + 1

            n_entries = self._queueloaders[mode].fetch_n_entries()

            if next_last_entry < n_entries:
                next_entries = numpy.arange(minibatch_size, dtype=numpy.int32) + last_entry + 1
            else:
                # Create an array to cover the entries till the last one ...
                next_entries_a = numpy.arange(n_entries - 1 - last_entry, dtype=numpy.int32) + last_entry + 1
                # ... and one array for the leftover entries, starting back from zero
                next_entries_b = numpy.arange((last_entry + 1 + minibatch_size) % n_entries, dtype=numpy.int32)
                # Finally concatenate the two arrays
                next_entries = numpy.concatenate((next_entries_a, next_entries_b))

        elif self._random_access == RandomAccess.random_blocks:
            # How many entries are there?
            n_entries = self._queueloaders[mode].fetch_n_entries()
            # The number of available choices is the number of entries - minibatch_size - 1:
            n_choices = n_entries - minibatch_size - 1
            start_entry = numpy.random.randint(low=0, high=n_choices, size=1)
            next_entries = numpy.arange(minibatch_size) + start_entry

        else:  # self._random_access == RandomAccess.random_events
            # Choose randomly, but require unique indexes:
            n_entries = self._queueloaders[mode].fetch_n_entries()
            next_entries = random.sample(range(n_entries), minibatch_size)


        self._queue_next_entries[mode] = next_entries

        return next_entries


#### This class is taken from queueloader.py. This python class is called from exec.py from larcv_fetcher.py, prepare_sample

In [11]:
class larcv_queueio (object):

    _instance_m={}

    @classmethod
    def exist(cls,name):
        name = str(name)
        return name in cls._instance_m

    @classmethod
    def instance_by_name(cls,name):
        return cls._instance_m[name]

    def __init__(self):
        self._proc = None
        self._name = ''
        self._verbose = False
        self._read_start_time = None
        self._read_end_time = None
        self._cfg_file = None
        self._storage = {}
        self._event_entries = None
        self._event_ids = None
        self._queues_set = False

    def reset(self):
        while self.is_reading(): time.sleep(0.01)
        if self._proc: self._proc.reset()

    def __del__(self):
        try:
            self.reset()
        except AttrbuteError:
            pass

    def configure(self,cfg, color=0):
        # if "this" was configured before, reset it
        if self._name: self.reset()

        # get name
        if not cfg['filler_name']:
            sys.stderr.write('filler_name is empty!\n')
            raise ValueError

        # ensure unique name
        if self.__class__.exist(cfg['filler_name']) and not self.__class__.instance_by_name(cfg['filler_name']) == self:
            sys.stderr.write('filler_name %s already running!' % cfg['filler_name'])
            return
        self._name = cfg['filler_name']

        # get QueueProcessor config file
        self._cfg_file = cfg['filler_cfg']
        # if not self._cfg_file or not os.path.isfile(self._cfg_file):
        #     sys.stderr.write('filler_cfg file does not exist: %s\n' % self._cfg_file)
        #     raise ValueError

        # set verbosity
        if 'verbosity' in cfg:
            self._verbose = bool(cfg['verbosity'])

        # configure thread processor
        self._proc = larcv.QueueProcessor(self._name)
        
        #I am testing here.....AB
        #okay Queue processor needs the configuration in the JSON format. _cfg_file is not 
        # in the JSON format. Need to add an additional layer of conversion.....AB
        #self._proc.configure(self._cfg_file, color)
        data = {}
        try:
            file_path = self._cfg_file
            with open(file_path, "r") as json_file:
                data = json.load(json_file)
        # Process the JSON data here
        except FileNotFoundError:
            print(f"queueloader.py:: The file '{file_path}' was not found.")
        except json.JSONDecodeError as e:
            print(f"queueloader.py:: Error decoding JSON: {e}")

        #self._proc.configure(self._cfg_file, color)
        self._proc.configure(data[self._name],color)

        # fetch batch filler info
        print("Batch fillers",self._proc.batch_fillers())
        self._storage = {}
        # Making a map between process names and data outputs:
        for pid, datatype in zip(self._proc.batch_fillers(), self._proc.batch_types()):
            # What's the name from the config?
            print("pid,datatype",pid,datatype)
            name = self._proc.storage_name(pid)
            datatype = larcv.BatchDataTypeName(datatype)
            self._storage[name]=batch_pydata(datatype)
            if 'make_copy' in cfg and cfg['make_copy']:
                self._storage[name]._make_copy = True


        # all success?
        # register *this* instance
        self.__class__._instance_m[self._name] = self


    def set_next_batch(self, batch_indexes):
        # if type(batch_indexes) != larcv.VectorOfSizet:
        #     indexes = larcv.VectorOfSizet()
        #     indexes.resize(len(batch_indexes))
        #     for i, val in enumerate(batch_indexes):
        #         indexes[i] = int(val)
        #     batch_indexes = indexes
        self._proc.set_next_batch(batch_indexes)

    def batch_process(self):
        while self.is_reading():
            time.sleep(0.01)
        self._proc.batch_process()


    def prepare_next(self):
        self._proc.prepare_next()


    def is_reading(self,storage_id=None):

        ready =  self._proc.is_next_ready() and not self._proc.is_reading()

        # print(f"Overall ready? {ready}")
        # print(f"QueueProcessor ready? {self._proc.is_next_ready()}")

        return not ready

    def pop_current_data(self):
        # Promote the "next" data to current in C++ and release current
        self._proc.pop_current_data()



    def next(self,store_entries=False,store_event_ids=False):

        # Calling next will load the next set of data into batch_pydata.  It does not do any
        # automatic data loading or steping, you must do this manually.

        for name,storage in self._storage.items():
            dtype = storage.dtype()
            batch_data = self._proc.__getattribute__(f"get_batch_{dtype}")(name)

            # print(type(batch_storage), " at ", batch_storage)
            # batch_data = batch_storage.get_batch()
            # print(type(batch_data), " at ", batch_data)
            storage.set_data(storage_id=name, larcv_batchdata=batch_data)

        if not store_entries: self._event_entries = None
        else: self._event_entries = self._proc.processed_entries()

        if not store_event_ids: self._event_ids = None
        else: self._event_ids = self._proc.processed_events()

        return

    def fetch_data(self,key):
        try:
            return self._storage[key]
        except KeyError:
            sys.stderr.write('Cannot fetch data w/ key %s (unknown)\n' % key)
            return

    def fetch_event_ids(self):
        return self._event_ids

    def fetch_entries(self):
        return self._event_entries

    def fetch_n_entries(self):
        return self._proc.get_n_entries()

#### This is the duplication of how larcv_queueio is called in larcv_fetcher. If you uncomment the following lines, you will reproduce the error. 

In [12]:
qio = larcv_queueio()
qio.configure(io_config)
print(io_config)
#print(qio._name)
#print(qio.fetch_entries())

Batch fillers []
{'filler_name': 'primary', 'filler_cfg': 'tmpsbbj5ext', 'verbosity': 5, 'make_copy': False}
    [NORMAL]  <IOManager::prepare_input> Opening a file in READ mode: "/Users/amitbashyal/SparseEventID/data/merged_sample_0.h5"
    [NORMAL]  <IOManager::prepare_input> File "/Users/amitbashyal/SparseEventID/data/merged_sample_0.h5" has 1404 entries
    [NORMAL]  <IOManager::initialize> Prepared input with 1404 entries...


### In fact these two lines pin point the source of errors. 

In [13]:
ids= qio.fetch_event_ids()

In [14]:
print(ids)
entries = qio.fetch_entries()
print(entries)
qio.next()
entries = qio.fetch_entries()
print(entries)

None
None
None


### The following code is taken from config_builder. This creates a json format config which seems to be expected by the IOManager.cxx in larcv

In [15]:
#This from larcv3/larcv/config_builder.py
import larcv
import json
import uuid

class ConfigBuilder:
    """
    Front-end to create configuration objects step by step.
    """


    valid_datatypes = \
        [ f"tensor{i+1}d" for i in range(4) ] + \
        [ f"sparse{i+2}d" for i in range(2) ] + \
        [ f"cluster{i+2}d" for i in range(2) ] + \
        [ f"bbox{i+2}d" for i in range(2) ] + \
        ["particle"] + \
        ["PID"]

    valid_preprocess = [
        "Downsample", "DenseToSparse", "Embed",
        "SparseToDense", "TensorFromCluster", "Threshold",
        "BBoxFromParticle", "BBoxFromCluster"
    ]

    def __init__(self):
        self.config = larcv.QueueProcessor.default_config()

    def get_config(self):
        """
        Gets the configuration.
        
        :returns:   The configuration.
        
        """
        return self.config

    def set_parameter(self,value,*keys):
        """
        Sets the parameter to value, walking into the config dictionary in the order of keys

        :param      value:  The value
        :type       value:  { type_description }
        :param      keys:   The keys
        :type       keys:   list

        """
        dct = self.config
        for key in keys[:-1]:
               dct = dct[key]
        dct[keys[-1]] = value
        return

    def print_config(self):
        """
        Prints a configuration using json to format it.
        
        """
        return json.dumps(self.config, indent=2)

    def validate_datatype(self, datatype : str):
        """
        Ensure datatype is viable

        :param      datatype:  The datatype
        :type       datatype:  str
        """

        if datatype not in self.valid_datatypes:
            raise TypeError(f"Unsupported datatype passed to ConfigBuilder: {datatype}")

    def validate_preprocess(self, preprocess : str):
        """
        Ensure preprocess is viable

        :param      preprocess:  The preprocess
        :type       preprocess:  str
        """

        if preprocess not in self.valid_preprocess:
            raise TypeError(f"Unsupported preprocess passed to ConfigBuilder: {preprocess}")


    def add_preprocess(self, datatype : str, producer : str, process : str, name : str = "", **kwargs):
        """
        Adds a preprocess.

        :param      datatype:  The datatype of the input
        :type       datatype:  str
        :param      producer:  The producer of the input
        :type       producer:  str
        :param      process:   The process name
        :type       process:   str
        :param      name:      The name of the process in the config (optional)
        :type       name:      str
        :param      kwargs:    The keywords arguments to override and parameters in the preprocess
        :type       kwargs:    dictionary
        """

        # Validate the datatype:
        self.validate_datatype(datatype)

        # Next, validate the preprocess:
        self.validate_preprocess(process)

        # Next, we get the default config for the requested process:
        proc_config = larcv.__getattribute__(process).default_config()

        # for all of the keyword args, make sure they are in the config:
        for kwarg in kwargs:
            if kwarg != "Profile" and kwarg not in proc_config:
                raise Exception(f"Config Parameter {kwarg} not available for"
                                f"{process}, available parameters: "
                                f"{proc_config.keys()}")

            # Override default configs:
            proc_config[kwarg] = kwargs[kwarg]

        # if the supplied name is empty, autogenerate a name:
        if name == "":
            name = f"{process}_{producer}_{datatype}_" + str(uuid.uuid4())[:8]

        proc_config["Producer"] = producer
        proc_config["Product"]  = datatype

        # Finally add this process to the configuration

        # Process Type is the 'process'
        self.config["ProcessDriver"]["ProcessType"].append(process)
        self.config["ProcessDriver"]["ProcessName"].append(name)

        if self.config["ProcessDriver"]["ProcessList"] is None:
            self.config["ProcessDriver"]["ProcessList"] = { name : proc_config }
        else:
            self.config["ProcessDriver"]["ProcessList"].update(
                {name : proc_config}
            )

        return


    def select_filler(self, datatype : str):
        """
        Pick the batch filler base on datatype

        :param      datatype:   The datatype
        :type       datatype:   str

        """

        if datatype == "tensor2d": return "BatchFillerTensor2D"
        elif datatype == "tensor3d": return "BatchFillerTensor3D"
        elif datatype == "sparse2d": return "BatchFillerSparseTensor2D"
        elif datatype == "sparse3d": return "BatchFillerSparseTensor3D"
        elif datatype == "bbox2d": return "BatchFillerBBox2D"
        elif datatype == "bbox3d": return "BatchFillerBBox3D"
        elif datatype == "PID": return "BatchFillerPIDLabel"
        elif datatype == "particle": return "BatchFillerParticle"
        else:
            raise Exception(f"Batch Filler not found for datatype{datatype}")


    def add_batch_filler(self, datatype : str, producer : str, name : str = "", **kwargs):
        """
        Adds a preprocess.

        :param      datatype:  The datatype to load into memory
        :type       datatype:  str
        :param      producer:  The producer of the datatype
        :type       producer:  str
        :param      name:      The name of the process in the config file (optional)
        :type       name:      str
        :param      kwargs:    Any override arguments to the configuration
        :type       kwargs:    dictionary
        """

        # Similar to adding a preprocess, but the process is uniquely ID'd
        # by the datatype

        # Validate the datatype:
        self.validate_datatype(datatype)

        # Next, validate the preprocess:
        filler = self.select_filler(datatype)

        # Next, we get the default config for the requested process:
        proc_config = larcv.__getattribute__(filler).default_config()

        # for all of the keyword args, make sure they are in the config:
        for kwarg in kwargs:
            if kwarg != "Profile" and kwarg not in proc_config:
                raise Exception(f"Config Parameter {kwarg} not available for"
                                f"{filler}, available parameters: "
                                f"{proc_config.keys()}")

            # Override default configs:
            proc_config[kwarg] = kwargs[kwarg]

        # if the supplied name is empty, autogenerate a name:
        if name == "":
            name = f"{filler}_{producer}_" + str(uuid.uuid4())[:8]

        proc_config["Producer"] = producer

        # Finally add this process to the configuration

        # Process Type is the 'process'
        self.config["ProcessDriver"]["ProcessType"].append(filler)
        self.config["ProcessDriver"]["ProcessName"].append(name)

        if self.config["ProcessDriver"]["ProcessList"] is None:
            self.config["ProcessDriver"]["ProcessList"] = { name : proc_config }
        else:
            self.config["ProcessDriver"]["ProcessList"].update(
                {name : proc_config}
            )

        return


In [16]:
cfg_builder = ConfigBuilder()

In [17]:
cfg_builder.print_config()
m_config = cfg_builder.get_config()

In [18]:
print(m_config)

{'InputFiles': [], 'ProcessDriver': {'EnableFilter': False, 'IOManager': {'IOMode': 0, 'Input': {'InputFiles': [], 'ReadOnlyName': [], 'ReadOnlyType': [], 'UseH5CoreDriver': False}, 'Output': {'Compression': 1, 'OutFileName': '', 'StoreOnlyName': [], 'StoreOnlyType': []}, 'Verbosity': 0}, 'NumEntries': 0, 'ProcessList': None, 'ProcessName': [], 'ProcessType': [], 'RandomAccess': False, 'RandomSeed': 0, 'StartEntry': 0, 'Verbosity': 0}, 'ProcessName': 0, 'Verbosity': 2}


In [19]:
for key in m_config:
    print(key)

InputFiles
ProcessDriver
ProcessName
Verbosity


In [20]:
print(m_config['ProcessDriver'])

{'EnableFilter': False, 'IOManager': {'IOMode': 0, 'Input': {'InputFiles': [], 'ReadOnlyName': [], 'ReadOnlyType': [], 'UseH5CoreDriver': False}, 'Output': {'Compression': 1, 'OutFileName': '', 'StoreOnlyName': [], 'StoreOnlyType': []}, 'Verbosity': 0}, 'NumEntries': 0, 'ProcessList': None, 'ProcessName': [], 'ProcessType': [], 'RandomAccess': False, 'RandomSeed': 0, 'StartEntry': 0, 'Verbosity': 0}


In [21]:
m_config['InputFiles'].append("/Users/amitbashyal/SparseEventID/data/merged_sample_0.h5")
#qio.configure(io_config)

In [22]:
print(io_config)

{'filler_name': 'primary', 'filler_cfg': 'tmpsbbj5ext', 'verbosity': 5, 'make_copy': False}


In [23]:
_proc = larcv.QueueProcessor("primary")

### The line below shows that QueueProcessor::configure takes the format produced from configbuilder class

In [24]:
_proc.configure(m_config)

      [INFO]  <ProcessDriver::configure::L124> Configuring IO
      [INFO]  <ProcessDriver::configure::L127> Retrieving self (ProcessDriver) config
      [INFO]  <ProcessDriver::configure::L130> Enable Filter is :  0
      [INFO]  <ProcessDriver::configure::L132> RandomAccess is :  0
      [INFO]  <ProcessDriver::configure::L149> Start looping process list to instantiate processes
     [DEBUG]  <ProcessDriver::override_input_file> ProcessDriver.cxx::L31 Called
    [NORMAL]  <primary::configure> Done configuration ...
     [DEBUG]  <ProcessDriver::process_names> ProcessDriver.cxx::L54 Called
     [DEBUG]  <ProcessDriver::initialize> ProcessDriver.cxx::L185 Called
      [INFO]  <ProcessDriver::initialize::L195> Initializing IO 
     [DEBUG]  <IOManager::initialize> IOManager.cxx::L143 start
     [DEBUG]  <IOManager::prepare_input> IOManager.cxx::L423 start
      [INFO]  <IOManager::prepare_input::L460> Start inspecting 1files
    [NORMAL]  <IOManager::prepare_input> Opening a file in REA

In [25]:
print(m_config)

{'InputFiles': ['/Users/amitbashyal/SparseEventID/data/merged_sample_0.h5'], 'ProcessDriver': {'EnableFilter': False, 'IOManager': {'IOMode': 0, 'Input': {'InputFiles': [], 'ReadOnlyName': [], 'ReadOnlyType': [], 'UseH5CoreDriver': False}, 'Output': {'Compression': 1, 'OutFileName': '', 'StoreOnlyName': [], 'StoreOnlyType': []}, 'Verbosity': 0}, 'NumEntries': 0, 'ProcessList': None, 'ProcessName': [], 'ProcessType': [], 'RandomAccess': False, 'RandomSeed': 0, 'StartEntry': 0, 'Verbosity': 0}, 'ProcessName': 0, 'Verbosity': 2}


In [26]:
print(io_config)

{'filler_name': 'primary', 'filler_cfg': 'tmpsbbj5ext', 'verbosity': 5, 'make_copy': False}


In [27]:
#qio.configure(io_config)

In [28]:
print(io_config)

{'filler_name': 'primary', 'filler_cfg': 'tmpsbbj5ext', 'verbosity': 5, 'make_copy': False}


In [29]:
print(m_config)

{'InputFiles': ['/Users/amitbashyal/SparseEventID/data/merged_sample_0.h5'], 'ProcessDriver': {'EnableFilter': False, 'IOManager': {'IOMode': 0, 'Input': {'InputFiles': [], 'ReadOnlyName': [], 'ReadOnlyType': [], 'UseH5CoreDriver': False}, 'Output': {'Compression': 1, 'OutFileName': '', 'StoreOnlyName': [], 'StoreOnlyType': []}, 'Verbosity': 0}, 'NumEntries': 0, 'ProcessList': None, 'ProcessName': [], 'ProcessType': [], 'RandomAccess': False, 'RandomSeed': 0, 'StartEntry': 0, 'Verbosity': 0}, 'ProcessName': 0, 'Verbosity': 2}


In [30]:
import yaml

In [31]:
def read_mixed_file(filename):
    # Read file as a string
    with open(filename, 'r') as f:
        content = f.read()
    
    # Convert to a format that's compliant with JSON
    content = content.replace("{", "{\n").replace("}", "\n}")
    content = content.replace(":", ": ").replace(", ", ",\n")
    lines = content.split('\n')
    lines = ['"' + line.split(":")[0].strip() + '":' + line.split(":")[1] if ':' in line else line for line in lines]
    json_compliant_content = "\n".join(lines)
    
    # Load the string as JSON
    data = json.loads(json_compliant_content)
    return data

#if __name__ == '__main__':
#    filename = '/tmp/tmpsbbj5ext'  # Replace with your file path
#    data = read_mixed_file(filename)
#    print(data)


In [32]:
cfg_builder.print_config()

'{\n  "InputFiles": [\n    "/Users/amitbashyal/SparseEventID/data/merged_sample_0.h5"\n  ],\n  "ProcessDriver": {\n    "EnableFilter": false,\n    "IOManager": {\n      "IOMode": 0,\n      "Input": {\n        "InputFiles": [],\n        "ReadOnlyName": [],\n        "ReadOnlyType": [],\n        "UseH5CoreDriver": false\n      },\n      "Output": {\n        "Compression": 1,\n        "OutFileName": "",\n        "StoreOnlyName": [],\n        "StoreOnlyType": []\n      },\n      "Verbosity": 0\n    },\n    "NumEntries": 0,\n    "ProcessList": null,\n    "ProcessName": [],\n    "ProcessType": [],\n    "RandomAccess": false,\n    "RandomSeed": 0,\n    "StartEntry": 0,\n    "Verbosity": 0\n  },\n  "ProcessName": 0,\n  "Verbosity": 2\n}'

In [33]:
config = larcv.QueueProcessor.default_config()

In [34]:
print(config)

{'InputFiles': [], 'ProcessDriver': {'EnableFilter': False, 'IOManager': {'IOMode': 0, 'Input': {'InputFiles': [], 'ReadOnlyName': [], 'ReadOnlyType': [], 'UseH5CoreDriver': False}, 'Output': {'Compression': 1, 'OutFileName': '', 'StoreOnlyName': [], 'StoreOnlyType': []}, 'Verbosity': 0}, 'NumEntries': 0, 'ProcessList': None, 'ProcessName': [], 'ProcessType': [], 'RandomAccess': False, 'RandomSeed': 0, 'StartEntry': 0, 'Verbosity': 0}, 'ProcessName': 0, 'Verbosity': 2}


### IO Config Content : 
{'filler_name': 'primary', 'filler_cfg': '/tmp/tmpsbbj5ext', 'verbosity': 5, 'make_copy': False}

### Config from QueueProcessor.cxx (More info in QueueProcessor::configure(const json&, int )
{'InputFiles': [], 'ProcessDriver': {'EnableFilter': False, 'IOManager': {'IOMode': 0, 'Input': {'InputFiles': [], 'ReadOnlyName': [], 'ReadOnlyType': [], 'UseH5CoreDriver': False}, 'Output': {'Compression': 1, 'OutFileName': '', 'StoreOnlyName': [], 'StoreOnlyType': []}, 'Verbosity': 0}, 'NumEntries': 0, 'ProcessList': None, 'ProcessName': [], 'ProcessType': [], 'RandomAccess': False, 'RandomSeed': 0, 'StartEntry': 0, 'Verbosity': 0}, 'ProcessName': 0, 'Verbosity': 2}

In [35]:
file_path = '/tmp/tmpsbbj5ext'
with open(file_path,'r') as f:
    file_content = f.read()
    
json_output = 'out.json'
with open(json_output,'w') as json_file:
    json.dump(file_content,json_file,indent=4)


In [36]:
with open(json_output,'r') as jfile:
    data = json.load(jfile)
print(json.dumps(data,indent=4))

"primary: {\n  Verbosity: 5 \n  EnableFilter: false \n  NumThreads: 4 \n  InputFiles: [\"/Users/amitbashyal/SparseEventID/data/merged_sample_0.h5\"]\n  NumBatchStorage: 4 \n  RandomSeed: 0 \n  RandomAccess: 0 \n  ProcessType: [\"BatchFillerSparseTensor3D\",\"BatchFillerPIDLabel\",\"BatchFillerPIDLabel\",\"BatchFillerPIDLabel\",\"BatchFillerPIDLabel\"]\n  ProcessName: [\"data\",\"label_neut\",\"label_prot\",\"label_cpi\",\"label_npi\"]\n\n  ProcessList: {\n    data: {\n      Verbosity: 3\n      TensorProducer: \"dunevoxels\"\n      IncludeValues: true\n      MaxVoxels: 30000\n      UnfilledVoxelValue: -999\n      Augment: true\n    }\n    label_neut: {\n      Verbosity: 3\n      ParticleProducer: neutID\n      PdgClassList: [0,1,2]\n    }\n    label_prot: {\n      Verbosity: 3\n      ParticleProducer: protID\n      PdgClassList: [0,1,2]\n    }\n    label_cpi: {\n      Verbosity: 3\n      ParticleProducer: cpiID\n      PdgClassList: [0,1]\n    }\n    label_npi: {\n      Verbosity: 3\n   

In [37]:
#_proc.configure(json_output)

### If everything had worked:

[NORMAL]  <primary::configure> Done configuration ...
    [NORMAL]  <IOManager::prepare_input> Opening a file in READ mode: "/Users/amitbashyal/SparseEventID/data/merged_sample_0.h5"
    [NORMAL]  <IOManager::prepare_input> File "/Users/amitbashyal/SparseEventID/data/merged_sample_0.h5" has 1404 entries
    [NORMAL]  <IOManager::initialize> Prepared input with 1404 entries...

In [38]:
j_data = json.dumps(data,indent=4)

In [39]:
m_config["InputFiles"]

['/Users/amitbashyal/SparseEventID/data/merged_sample_0.h5']

In [40]:
config_json = json.loads(j_data)

In [41]:
print(type(m_config),type(config_json))

<class 'dict'> <class 'str'>


In [42]:
file_path = '/tmp/tmpsbbj5ext'
with open(file_path,'r') as f:
    file_content = f.read()
    #data = json.load(f)
    print(file_content)

primary: {
  Verbosity: 5 
  EnableFilter: false 
  NumThreads: 4 
  InputFiles: ["/Users/amitbashyal/SparseEventID/data/merged_sample_0.h5"]
  NumBatchStorage: 4 
  RandomSeed: 0 
  RandomAccess: 0 
  ProcessType: ["BatchFillerSparseTensor3D","BatchFillerPIDLabel","BatchFillerPIDLabel","BatchFillerPIDLabel","BatchFillerPIDLabel"]
  ProcessName: ["data","label_neut","label_prot","label_cpi","label_npi"]

  ProcessList: {
    data: {
      Verbosity: 3
      TensorProducer: "dunevoxels"
      IncludeValues: true
      MaxVoxels: 30000
      UnfilledVoxelValue: -999
      Augment: true
    }
    label_neut: {
      Verbosity: 3
      ParticleProducer: neutID
      PdgClassList: [0,1,2]
    }
    label_prot: {
      Verbosity: 3
      ParticleProducer: protID
      PdgClassList: [0,1,2]
    }
    label_cpi: {
      Verbosity: 3
      ParticleProducer: cpiID
      PdgClassList: [0,1]
    }
    label_npi: {
      Verbosity: 3
      ParticleProducer: npiID
      PdgClassList: [0,1]
    }
  }

In [43]:
print(json.dumps(m_config,indent=4))

{
    "InputFiles": [
        "/Users/amitbashyal/SparseEventID/data/merged_sample_0.h5"
    ],
    "ProcessDriver": {
        "EnableFilter": false,
        "IOManager": {
            "IOMode": 0,
            "Input": {
                "InputFiles": [],
                "ReadOnlyName": [],
                "ReadOnlyType": [],
                "UseH5CoreDriver": false
            },
            "Output": {
                "Compression": 1,
                "OutFileName": "",
                "StoreOnlyName": [],
                "StoreOnlyType": []
            },
            "Verbosity": 0
        },
        "NumEntries": 0,
        "ProcessList": null,
        "ProcessName": [],
        "ProcessType": [],
        "RandomAccess": false,
        "RandomSeed": 0,
        "StartEntry": 0,
        "Verbosity": 0
    },
    "ProcessName": 0,
    "Verbosity": 2
}


In [44]:
import json

file_path = "tmpsbbj5ext"

try:
    with open(file_path, "r") as json_file:
        data = json.load(json_file)
    # Process the JSON data here
except FileNotFoundError:
    print(f"The file '{file_path}' was not found.")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")


In [45]:
#_proc.configure(data["primary"])

In [46]:
print(data["primary"])

{'Verbosity': 5, 'EnableFilter': False, 'NumThreads': 4, 'InputFiles': ['/Users/amitbashyal/SparseEventID/data/merged_sample_0.h5'], 'NumBatchStorage': 4, 'RandomSeed': 0, 'RandomAccess': 0, 'ProcessType': ['BatchFillerSparseTensor3D', 'BatchFillerPIDLabel', 'BatchFillerPIDLabel', 'BatchFillerPIDLabel', 'BatchFillerPIDLabel'], 'ProcessName': ['data', 'label_neut', 'label_prot', 'label_cpi', 'label_npi'], 'ProcessList': {'data': {'Verbosity': 3, 'TensorProducer': 'dunevoxels', 'IncludeValues': True, 'MaxVoxels': 30000, 'UnfilledVoxelValue': -999, 'Augment': True}, 'label_neut': {'Verbosity': 3, 'ParticleProducer': 'neutID', 'PdgClassList': [0, 1, 2]}, 'label_prot': {'Verbosity': 3, 'ParticleProducer': 'protID', 'PdgClassList': [0, 1, 2]}, 'label_cpi': {'Verbosity': 3, 'ParticleProducer': 'cpiID', 'PdgClassList': [0, 1]}, 'label_npi': {'Verbosity': 3, 'ParticleProducer': 'npiID', 'PdgClassList': [0, 1]}}}


In [47]:
_proc.configure(data["primary"])

     [DEBUG]  <ProcessDriver::reset> ProcessDriver.cxx::L19 Called
     [DEBUG]  <IOManager::reset> IOManager.cxx::L1102 start
     [DEBUG]  <ProcessDriver::reset> ProcessDriver.cxx::L19 Called
     [DEBUG]  <IOManager::reset> IOManager.cxx::L1102 start
     [DEBUG]  <ProcessDriver::reset> ProcessDriver.cxx::L19 Called
     [DEBUG]  <IOManager::reset> IOManager.cxx::L1102 start
      [INFO]  <ProcessDriver::configure::L124> Configuring IO
      [INFO]  <ProcessDriver::configure::L127> Retrieving self (ProcessDriver) config
      [INFO]  <ProcessDriver::configure::L130> Enable Filter is :  0
      [INFO]  <ProcessDriver::configure::L132> RandomAccess is :  0
      [INFO]  <ProcessDriver::configure::L149> Start looping process list to instantiate processes
     [DEBUG]  <ProcessDriver::override_input_file> ProcessDriver.cxx::L31 Called
     [DEBUG]  <ProcessDriver::process_names> ProcessDriver.cxx::L54 Called
     [DEBUG]  <ProcessDriver::initialize> ProcessDriver.cxx::L185 Called
      

In [48]:
_proc.prepare_next()